In [ ]:
import os
import re

import pandas as pd
import requests
from prettyprinter import cpprint

In [ ]:
def decode_html(line):
    line = line.replace("&quot;", '"')
    line = line.replace("&#8230;", "…")
    line = line.replace("&amp;", "&")
    line = line.replace("''", '"')

    return line

In [ ]:
target_url = "http://ja.scp-wiki.net/members-pages-jp"
start_word = '<h2 id="toc0"><span>管理者</span></h2>'
end_word = '<h2><span>著者ページへの参加条件</span></h2>'

In [ ]:
response = requests.get(target_url)
if response.status_code is not requests.codes.ok:
            print(f"{key} request err : {response.status_code}")

In [ ]:
lines = response.text.split("\n")
start = lines.index(start_word)

In [209]:
urls = []
titles = []
authers = []
brts = []
images = []


for line in lines[start:]:
    auther = ""
    title = ""
    url = ""
    image = ""

    if end_word in line:
        break
    elif '<td><span class="printuser avatarhover">' in line: 
        line = decode_html(line)
        sp_line = re.split('[<>]', line)
        sp_line[7] = sp_line[7].replace('img class="small" src="', "").split()[0]
        auther = sp_line[12].replace('"',"")
        image = sp_line[7].replace('"',"")
        authers.append(auther)
        images.append(image)
        
    elif '<td><span class="error-inline"><em>' in line:
        line = decode_html(line)
        sp_line = re.split('[<>]', line)
        auther = sp_line[6]
        authers.append(auther)
        images.append(image)

    
    elif '<td><span' in line:
        line = decode_html(line)
        sp_line = re.split('[<>]', line)
        url = sp_line[7].replace("a href=", "").replace('"',"")
        title = "~~" + sp_line[4] + "~~" + sp_line[8]
        urls.append(url)
        titles.append(title)
        brts.append("jp")
        
    elif '<td><a style' in line:
        line = decode_html(line)
        sp_line = re.split('[<>]', line)
        url = sp_line[3][sp_line[3].find("/author"):].replace('"', '')
        
        print(sp_line[4])
        title = sp_line[4]
        urls.append(url)
        titles.append(title)
        brts.append("jp")

    elif '<td><a href="/author' in line:
        line = decode_html(line)
        sp_line = re.split('[<>]', line)
        url = sp_line[3].replace("a href=", "").replace('"',"")
        title = sp_line[4].replace('"',"")
        urls.append(url)
        titles.append(title)
        brts.append("jp")
        
    
    elif 'No memberPage' in line:
        line = decode_html(line)
        sp_line = re.split('[<>]', line)
        title = sp_line[2]
        urls.append("")
        titles.append(title)
        brts.append("jp")
        
    else:
        continue  

unQua=fied - No m=berPage
"Dr.


In [210]:
df = pd.DataFrame(columns=['url', 'title', 'auther', 'branches', 'images'])

In [216]:
print(urls[len(urls)-1:])
print(titles[len(titles) -1 :])
print(authers[len(authers) -1:])
print(brts[len(brts) - 1:])

['/author:soya-one']
['磯日井博士の人事ファイル']
['SOYA_one']
['jp']


In [141]:
df['url'] = urls
df['title'] = titles
df['auther'] = authers
df['branches'] = brts

ValueError: Length of values does not match length of index

In [148]:
titles

['エージェント・東風浦の人事ファイル',
 '鬼食料理長の人事ファイル',
 '味 研究員の人事ファイル',
 'エージェント・ユーリィの人事ファイル',
 'カメラマン・甘梨の人事ファイル',
 'ただの果樹園',
 'unQualified - No memberPage',
 'デバンの作者ページ',
 '般若博士兼エージェントの人事ファイル',
 'PoIデータファイル ZI-0 おめんライダーあんず',
 '日野博士の人事ファイル',
 '三国技師の人事ファイル',
 'エージェント・育良の人事ファイル',
 'izhayaの著者ページ',
 'エージェント・イヴァノフの人事ファイル',
 '玉菜博士の人事ファイル',
 'KoikeSanshouemonの著者ページ',
 '秋津洲博士の人事ファイル',
 '南方博士の人事ファイル',
 'エージェント・海野の人事ファイル',
 '桜野姉妹+1の人事ファイル',
 'unQualified - No memberPage',
 '河上研究員の人事ファイル',
 'エージェント魚住の人事ファイル',
 '失敗剥製白熊',
 'vハ国d5天ka5%ッ行oブへus瓶ftの',
 '宇喜田博士の人事ファイル',
 'unQualified - No memberPage',
 '黒埼会計主任の人事ファイル',
 '7happy7の著者ページ',
 'バクスター博士の人事ファイル',
 '本宮交渉人補佐/本宮研究員の人事ファイル',
 'ak1-yorunagaの著者ページ',
 '知里研究員の人事ファイル',
 '霧甲水博士の人事ファイル',
 '白鳥研究員の人事ファイル',
 '甘ヶ崎研究員の人事ファイル',
 '天見補助員の人事ファイル',
 'AI神州の人事ファイル',
 '壊れた翻訳箱',
 '朝比奈博士の人事ファイル',
 '阿藤博士の人事ファイル',
 '氷菓監査官の時計塔',
 '串間保育士の人事ファイル',
 '不条博士の人事ファイル',
 'クレイドル＝ノヴォス.AICの人事ファイル',
 'エージェント・ヤマトモの人事ファイル',
 '左衛門三郎博士の人事ファイル',
 "エージェンツ・田中'sの人事ファイル",
 '財団法人SeCaPer或いは本日も良いカブトムシ収穫日和です。',
 'C